In [ ]:
import math
import scipy as sp
from scipy.stats import bernoulli, uniform
import numpy as np
from numpy.testing import assert_allclose
import json
np.random.seed(123456789)

In [ ]:
# !pip install rilacs
from rilacs.strategies import linear_gamma_dist
import pytest
from rilacs.martingales import (
    apriori_Kelly_martingale,
    distKelly_martingale,
    sqKelly_martingale,
    dKelly_martingale,
)
import itertools

In [ ]:
# to save dicts as json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
def sprt_mart(x : np.array, N : int, mu : float=1/2, eta: float=1-np.finfo(float).eps, \
              u: float=1, random_order = True):
    '''
    Finds the p value for the hypothesis that the population 
    mean is less than or equal to mu against the alternative that it is eta,
    for a population of size N of values in the interval [0, u].
    
    Generalizes Wald's SPRT for the Bernoulli to sampling without replacement and to bounded
    values rather than binary values.

    If N is finite, assumes the sample is drawn without replacement
    If N is infinite, assumes the sample is with replacement
    
    Data are assumed to be in random order. If not, the calculation for sampling without replacement is incorrect.


    
    Parameters:
    -----------
    x : binary list, one element per draw. A list element is 1 if the 
        the corresponding trial was a success
    N : int
        population size for sampling without replacement, or np.infinity for 
        sampling with replacement
    theta : float in (0,u)
        hypothesized population mean
    eta : float in (0,u)
        alternative hypothesized population mean
    random_order : Boolean
        if the data are in random order, setting this to True can improve the power.
        If the data are not in random order, set to False
    '''
    if any((xx < 0 or xx > u) for xx in x):
        raise ValueError(f'Data out of range [0,{u}]')
    if np.isfinite(N):
        if not random_order:
            raise ValueError("data must be in random order for samples without replacement")
        S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
        j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
        m = (N*mu-S)/(N-j+1)                   # mean of population after (j-1)st draw, if null is true
    else:
        m = mu
    with np.errstate(divide='ignore',invalid='ignore'): 
        terms = np.cumprod((x*eta/m + (u-x)*(u-eta)/(u-m))/u) # generalization of Bernoulli SPRT
    terms[m<0] = np.inf                        # the null is surely false
    return terms


In [ ]:
def shrink_trunc(x: np.array, N: int, mu: float=1/2, nu: float=1-np.finfo(float).eps, u: float=1, c: float=1/2, 
                 d: float=100) -> np.array: 
    '''
    apply the shrinkage and truncation estimator to an array
    
    sample mean is shrunk towards nu, with relative weight d compared to a single observation.
    estimate is truncated above at u-u*eps and below at mu_j+e_j(c,j)
    
    S_1 = 0
    S_j = \sum_{i=1}^{j-1} x_i, j > 1
    m_j = (N*mu-S_j)/(N-j+1) if np.isfinite(N) else mu
    e_j = c/sqrt(d+j-1)
    eta_j =  ( (d*nu + S_j)/(d+j-1) \vee (m_j+e_j) ) \wedge u*(1-eps)
    
    Parameters
    ----------
    x : np.array
        input data       
    mu : float in (0, 1)
        hypothesized population mean
    eta : float in (t, 1)
        initial alternative hypothethesized value for the population mean
    c : positive float
        scale factor for allowing the estimated mean to approach t from above
    d : positive float
        relative weight of nu compared to an observation, in updating the alternative for each term
    '''
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true 
    return np.minimum(u*(1-np.finfo(float).eps), np.maximum((d*nu+S)/(d+j-1),m+c/np.sqrt(d+j-1)))

In [ ]:
def test_shrink_trunc():
    epsj = lambda c, d, j: c/math.sqrt(d+j-1)
    Sj = lambda x, j: 0 if j==1 else np.sum(x[0:j-1])
    muj = lambda N, mu, x, j: (N*mu - Sj(x, j))/(N-j+1) if np.isfinite(N) else mu
    nus = [.51, .55, .6]
    mu = 1/2
    u = 1
    d = 10
    vrand =  sp.stats.bernoulli.rvs(1/2, size=20)
    v = [
        np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]),
        np.array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0]),
        vrand
    ]
    for nu in nus:
        c = (nu-mu)/2
        for x in v:
            N = len(x)
            xinf = shrink_trunc(x, np.inf, mu, nu, c=c, d=d)
            xfin = shrink_trunc(x, len(x), mu, nu, c=c, d=d)
            yinf = np.zeros(len(x))
            yfin = np.zeros(len(x))
            for j in range(1,len(x)+1):
                est = (d*nu + Sj(x,j))/(d+j-1)
                most = u*(1-np.finfo(float).eps)
                yinf[j-1] = np.minimum(np.maximum(mu+epsj(c,d,j), est), most)
                yfin[j-1] = np.minimum(np.maximum(muj(N,mu,x,j)+epsj(c,d,j), est), most)
            np.testing.assert_allclose(xinf, yinf)    
            np.testing.assert_allclose(xfin, yfin)    
    
test_shrink_trunc()

In [ ]:
def alpha_mart(x: np.array, N: int, mu: float=1/2, eta: float=1-np.finfo(float).eps, u: float=1, \
               estim: callable=shrink_trunc) -> np.array :
    '''
    Finds the ALPHA martingale for the hypothesis that the population 
    mean is less than or equal to t using a martingale method,
    for a population of size N, based on a series of draws x.
    
    The draws must be in random order, or the sequence is not a martingale under the null
    
    If N is finite, assumes the sample is drawn without replacement
    If N is infinite, assumes the sample is with replacement

    Parameters
    ----------
    x : list corresponding to the data
    N : int
        population size for sampling without replacement, or np.infinity for sampling with replacement
    mu : float in (0,1)
        hypothesized fraction of ones in the population
    eta : float in (t,1) 
        alternative hypothesized population mean
    estim : callable
        estim(x, N, mu, eta, u) -> np.array of length len(x), the sequence of values of eta_j for ALPHA
               
    Returns
    -------   
    terms : array
        sequence of terms that would be a nonnegative martingale under the null
    '''
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true 
    etaj = estim(x, N, mu, eta, u) 
    with np.errstate(divide='ignore',invalid='ignore'):
        terms = np.cumprod((x*etaj/m + (1-x)*(u-etaj)/(u-m))/u)
    terms[m<0] = np.inf
    return terms

# Sampling with replacement

In [ ]:
# Calculations for Table 1: sampling with replacement

reps = int(10**3)
block_size = int(10**7) # number of Bernoulli RVs to generate as a batch
max_size = int(10**7)

thetal = [0.505, 0.51, 0.52, 0.53, 0.54, 0.55, 0.6, 0.65, 0.7]
etal = thetal
alpha = 0.05
mu = 1/2

c_base=0.5
dl=[10, 100, 500, 1000]

results_a = {}
results_w = {}
done_a = {}
done_w = {}
for theta in thetal:
    results_a[theta] = {}
    results_w[theta] = {}
    for eta in etal:
        results_a[theta][eta] = {}
        results_w[theta][eta] = 0
        done_a[eta] = {}
        for d in dl:
            results_a[theta][eta][d] = 0
    for i in range(reps):
        done = False
        x = np.array([])
        for eta in etal:
            done_w[eta] = False
            for d in dl:
                done_a[eta][d] = False
        while not done and len(x) < max_size:
            x = np.append(x, bernoulli.rvs(theta, size=block_size))
            for eta in etal:          
                c = c_base*(eta-1/2)
                for d in dl:
                    if not done_a[eta][d]:
                        mart = alpha_mart(x, np.inf, mu=1/2, eta=eta, u=1, \
                                     estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,1,c=c,d=d))
                        t = np.argmax(mart >= 1/alpha)
                        if t>0:
                            results_a[theta][eta][d] += t 
                            done_a[eta][d] = True
                if not done_w[eta]:
                    mart = np.cumprod(x*eta/mu+(1-x)*(1-eta)/(1-mu))
                    t = np.argmax(mart >= 1/alpha)
                    if t>0:
                        results_w[theta][eta] += t
                        done_w[eta] = True
            done = all([all(done_a[g].values()) for g in done_a.keys()]) and all(done_w.values())
        if not done:
            for eta in etal:
                results_w[theta][eta] += (np.inf if not done_w[eta] else 0)
                for d in dl:
                    results_a[theta][eta][d] += (np.inf if not done_a[eta][d] else 0)
    for eta in etal:
        results_w[theta][eta] = results_w[theta][eta]/reps + 1
        print(f'\n{theta=} {eta=} {results_w[theta][eta]=:.1f}')
        for d in dl:
            results_a[theta][eta][d] = results_a[theta][eta][d]/reps + 1
        print(f'{[f"{d}: {results_a[theta][eta][d]:.1f}" for d in dl]}')

In [ ]:
# encode dicts and save as json

file_stems = ['results_w','results_a']
for fs in file_stems:
    with open(fs+'-replace.json','w') as file:
        file.write(json.dumps(eval(fs), cls=NpEncoder, indent = 4))   

In [ ]:
for theta in thetal:
    print('\\hline')
    for eta in etal:
            print(f'{theta} & {eta} &'
                  f'{[(round(results_a[theta][eta][d]) if np.isfinite(results_a[theta][eta][d]) else "---" ) for d in dl]}'
                  f' & {round(results_w[theta][eta]) if np.isfinite(results_w[theta][eta]) else "---"} \\\\')

In [ ]:
print('\\begin{tabular}{l|r|rrrr|r} \\\\' + \
      '\cline{3-6} $\theta$ & $\eta$ & \\multicolumn{4}{|c|}{ALPHA $d=$} & BRAVO \\\\ \n' + \
       '\\hline & ' + f'''{" ".join([f"& {d :,.0f}" for d in dl])}''')

for theta in thetal:
      print('\\hline')
      for eta in etal:
          print(f'{theta} & {eta} ' + \
            f'{" ".join([f"{(results_a[theta][eta][d] if np.isfinite(results_a[theta][eta][d]) else '''---''' ) : ,.0f}" for d in dl])}' + \
            f'{" ".join(f"{(results_w[theta][eta] if np.isfinite(results_w[theta][eta]) else '''---''' ) : ,.0f}")}')

print('\\end{tabular}\\\\ \\caption{\protect \label{tab:replace} Mean sample size to reject the null that ' + \
      'the population mean $\theta$ is greater than 1/2 in sampling with replacement, for ALPHA with various ' + \
      'values of $\eta$ and $d$ and for BRAVO for the same values of $\eta$. ' + \
      f"Average over {reps :,.0f} simulations.  \`\`---\'\' signifies that in at least one simulation, the sample " + \
      f'size exceeded {max_size :,.0f}.' + '}\n')

# Sampling without replacement, no invalid votes

In [ ]:
# Calculations for Table 2, sampling without replacement
# N, sam_max, theta, and eta were chosen to allow comparison to Huang et al. 20

reps = int(10**5)

thetal = [.505, .51, .52, .55, .6, .64, .7]
N = 20000
sam_max = 2000
etal = [.51, .55, .7]  # used for BRAVO in Huang et al 2020
alpha = 0.05

# for ALPHA
c_base=0.5
dl= [10, 100, 500, 1000]

# for RiLACS
D = 10
beta = 1

resl = ['rej_sam','not_rej_sam','rej_N','not_rej_N']

results_a = {}
results_apa = {}
results_apk = {}
results_sqk = {}

for theta in thetal:
    print(f'{theta=}')
    n_A = int(N*theta)
    n_B = N - n_A
    x = np.array([1]*n_A+[0]*n_B)
    results_a[theta] = {}
    results_apa[theta] = {}
    results_apk[theta] = {}
    results_sqk[theta] = {}
    for r in resl:
        results_sqk[theta][r] = 0
    for eta in etal:
        results_apk[theta][eta] = {}
        results_apa[theta][eta] = {}
        for r in resl:
            results_apk[theta][eta][r] = 0
            results_apa[theta][eta][r] = 0
        results_a[theta][eta]={} 
        for d in dl:
            results_a[theta][eta][d] = {}
            for r in resl:
                results_a[theta][eta][d][r] = 0
    for i in range(reps):
        np.random.shuffle(x)
        
        # sqKelly
        mart = sqKelly_martingale(x, m=1/2, N=N, D=D, beta=beta)
        # rejections by sam_max
        found = np.argmax(mart[0:sam_max] >= 1/alpha)
        results_sqk[theta]['rej_sam'] += (found if found > 0 else N-1)     
        results_sqk[theta]['not_rej_sam'] += (found==0)
        # rejections by N
        found = np.argmax(mart >= 1/alpha)
        results_sqk[theta]['rej_N'] += found
        results_sqk[theta]['not_rej_N'] += (found==0) # should not occur   
        
        # a priori Kelly and a priori SPRT
        for eta in etal:
            n_eta_A = int(N*eta)
            n_eta_B = N - n_eta_A
            c = c_base*(eta-1/2)
            
            # a priori Kelly
            mart = apriori_Kelly_martingale(x, m=0.5, N=N, n_A=n_eta_A, n_B=n_eta_B)
            # rejections by sam_max 
            found = np.argmax(mart[0:sam_max] >= 1/alpha)
            results_apk[theta][eta]['rej_sam'] += (found if found > 0 else N-1)   
            results_apk[theta][eta]['not_rej_sam'] += (found==0)
            # rejections by N
            found = np.argmax(mart >= 1/alpha)
            results_apk[theta][eta]['rej_N'] += found
            results_apk[theta][eta]['not_rej_N'] += (found==0) # should not occur
            
            # a priori SPRT
            mart = sprt_mart(x, N, mu=1/2, eta=eta, u=1, random_order=True)
            # rejections by sam_max 
            found = np.argmax(mart[0:sam_max] >= 1/alpha)
            results_apa[theta][eta]['rej_sam'] += (found if found > 0 else N-1)       
            results_apa[theta][eta]['not_rej_sam'] += (found==0)
            # rejections by N
            found = np.argmax(mart >= 1/alpha)
            results_apa[theta][eta]['rej_N'] += found
            results_apa[theta][eta]['not_rej_N'] += (found==0) # should not occur        
            
            # ALPHA
            for d in dl:               
                mart = alpha_mart(x, N, mu=1/2, eta=eta, u=1, \
                                  estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,1,c=c,d=d))
                # rejections by sam_max 
                found = np.argmax(mart[0:sam_max] >= 1/alpha)
                results_a[theta][eta][d]['rej_sam'] += (found if found > 0 else N-1)       
                results_a[theta][eta][d]['not_rej_sam'] += (found==0)
                # rejections by N
                found = np.argmax(mart >= 1/alpha)
                results_a[theta][eta][d]['rej_N'] += found  
                results_a[theta][eta][d]['not_rej_N'] += (found==0) # should not occur        
                
for theta in thetal:
    results_sqk[theta]['rej_sam'] = results_sqk[theta]['rej_sam']/reps + 1
    results_sqk[theta]['not_rej_sam'] = results_sqk[theta]['not_rej_sam']/reps 
    results_sqk[theta]['rej_N'] = results_sqk[theta]['rej_N']/reps + 1
    if results_sqk[theta]['not_rej_N'] > 0:
        print(f'sqKelly did not reject for {theta=}')  # sanity check
        
    for eta in etal:
        results_apk[theta][eta]['rej_sam'] = results_apk[theta][eta]['rej_sam']/reps + 1
        results_apk[theta][eta]['not_rej_sam'] = results_apk[theta][eta]['not_rej_sam']/reps 
        results_apk[theta][eta]['rej_N'] = results_apk[theta][eta]['rej_N']/reps  + 1
        if results_apk[theta][eta]['not_rej_N'] > 0:
            print(f'a priori Kelly did not reject for {theta=}, {eta=}')

        results_apa[theta][eta]['rej_sam'] = results_apa[theta][eta]['rej_sam']/reps + 1
        results_apa[theta][eta]['not_rej_sam'] = results_apa[theta][eta]['not_rej_sam']/reps 
        results_apa[theta][eta]['rej_N'] = results_apa[theta][eta]['rej_N']/reps + 1
        if results_apa[theta][eta]['not_rej_N'] > 0:
            print(f'a priori ALPHA did not reject for {theta=}, {eta=}')
            
        for d in dl:
            results_a[theta][eta][d]['rej_sam'] = results_a[theta][eta][d]['rej_sam']/reps + 1
            results_a[theta][eta][d]['not_rej_sam'] = results_a[theta][eta][d]['not_rej_sam']/reps 
            results_a[theta][eta][d]['rej_N'] = results_a[theta][eta][d]['rej_N']/reps + 1
            if results_a[theta][eta][d]['not_rej_N'] > 0:
                print(f'ALPHA did not reject for {theta=}, {eta=}, {d=}')

In [ ]:
# encode dicts and save as json

file_stems = ['results_sqk','results_apk','results_apa','results_a']
for fs in file_stems:
    with open(fs+'.json','w') as file:
        file.write(json.dumps(eval(fs), cls=NpEncoder, indent = 4))   

In [ ]:
# test reading dicts from json
file_stems = ['results_sqk','results_apk','results_apa','results_a']
for fs in file_stems:
    with open(fs+'.json','r') as file:
        data = json.load(file)
        exec(fs + ' = data')

# NOTE: keys that were ints are now strings

In [ ]:
# NOTE: keys that were ints are now strings
print('\\begin{tabular}{ll|rrrrrrr|rrrrrrr}' +
      '& & \\multicolumn{7}{|c|}{$n=2,000$} &  \multicolumn{7}{|c}{$n=N$} \\\\ \n' +
      'Method & & \\multicolumn{7}{c|}{mean sample size, $\\theta=$} ' +
      ' & \\multicolumn{7}{|c}{mean sample size, $\\theta=$ }\\\\ \n' +
      f'& {" ".join((*["& .505 & .51 & .52 & .55 & .6 & .64 & .7 "]*2,))} \\\\ \n' +
      '\\hline')

print('sqKelly  & ' +
      f''' {" ".join([f"& {int(results_sqk[str(theta)]['rej_sam']) :,.0f} " for theta in thetal])} ''' +
      f''' {" ".join([f"& {int(results_sqk[str(theta)]['rej_N']) :,.0f} " for theta in thetal])} \\\\ '''
      )
print('\\hline')
for eta in etal:
    print(f'''a priori Kelly & $\\eta={eta}$  ''' +
          f''' {" ".join([f"& {int(results_apk[str(theta)][str(eta)]['rej_sam']) :,.0f} " for theta in thetal])} ''' +
          f''' {" ".join([f"& {int(results_apk[str(theta)][str(eta)]['rej_N']) :,.0f} " for theta in thetal])} \\\\ '''
          )
print('\\hline')
for eta in etal:
    print(f'''SPRT & $\\eta={eta}$ ''' +
          f''' {" ".join([f"& {int(results_apa[str(theta)][str(eta)]['rej_sam']):,.0f} " for theta in thetal])} ''' +
          f''' {" ".join([f"& {int(results_apa[str(theta)][str(eta)]['rej_N']) :,.0f} " for theta in thetal])} \\\\  '''
          )
print('\\hline')
for eta in etal:
    for d in dl:
        print(f'''ALPHA & $\\eta={eta}$ $d={d :,.0f}$ ''' +
              f'''{" ".join([f"& {int(results_a[str(theta)][str(eta)][str(d)]['rej_sam']) :,.0f} " for theta in thetal])} ''' +
              f'''{" ".join([f"& {int(results_a[str(theta)][str(eta)][str(d)]['rej_N']) :,.0f} " for theta in thetal])} \\\\ '''
          )        
print('\\end{tabular} \n')


## Sampling without replacement with some non-votes

In [ ]:
# Tables 3 and 4

reps = int(10**3)

thetal = [.51, .52, .55, .6, .7]
blanks = [.1, .25, .5, .75]
Nl = [10000, 100000, 500000]
etal = thetal  
alpha = 0.05

# for ALPHA
c_base=0.5
dl= [10, 100, 1000]

# for RiLACs
D = 10
beta = 1

resl = ['rej_N','not_rej_N']

results_a_b = {}
results_apa_b = {}
results_apk_b = {}
results_sqk_b = {}

for theta in thetal:
    print(f'{theta=}')
    results_a_b[theta] = {}
    results_apa_b[theta] = {}
    results_apk_b[theta] = {}
    results_sqk_b[theta] = {}
    for N in Nl:
        print(f'\t{N=}')
        results_a_b[theta][N] = {}
        results_apa_b[theta][N] = {}
        results_apk_b[theta][N] = {}
        results_sqk_b[theta][N] = {}
        for b in blanks:
            print(f'\t\tblanks={b}')
            results_a_b[theta][N][b] = {}
            results_apa_b[theta][N][b] = {}
            results_apk_b[theta][N][b] = {}
            results_sqk_b[theta][N][b] = {}
            non_blank = int(N*(1-b))
            n_A = int(non_blank*theta)
            n_B = non_blank - n_A
            x = np.array([1]*n_A+[0]*n_B+[0.5]*(N-non_blank))
            for r in resl:
                results_sqk_b[theta][N][b][r] = 0
            for eta in etal:
                results_apk_b[theta][N][b][eta] = {}
                results_apa_b[theta][N][b][eta] = {}
                for r in resl:
                    results_apk_b[theta][N][b][eta][r] = 0
                    results_apa_b[theta][N][b][eta][r] = 0
                results_a_b[theta][N][b][eta]={} 
                for d in dl:
                    results_a_b[theta][N][b][eta][d] = {}
                    for r in resl:
                        results_a_b[theta][N][b][eta][d][r] = 0
            for i in range(reps):
                np.random.shuffle(x)

                # sqKelly
                mart = sqKelly_martingale(x, m=1/2, N=N, D=D, beta=beta)
                # rejections by N
                found = np.argmax(mart >= 1/alpha)
                results_sqk_b[theta][N][b]['rej_N'] += found
                results_sqk_b[theta][N][b]['not_rej_N'] += (found==0) # should not occur   

                # a priori Kelly and a priori SPRT
                for eta in etal:
                    n_eta_A = int(non_blank*eta)
                    n_eta_B = non_blank - n_eta_A
                    eta_shangrla = (non_blank*eta + (N-non_blank)/2)/N
                    c = c_base*(eta-1/2)

                    # a priori Kelly
                    mart = apriori_Kelly_martingale(x, m=0.5, N=N, n_A=n_eta_A, n_B=n_eta_B)
                    # rejections by N
                    found = np.argmax(mart >= 1/alpha)
                    results_apk_b[theta][N][b][eta]['rej_N'] += found
                    results_apk_b[theta][N][b][eta]['not_rej_N'] += (found==0) # should not occur

                    # a priori SPRT
                    mart = sprt_mart(x, N, mu=1/2, eta=eta_shangrla, u=1, random_order=True)
                    # rejections by N
                    found = np.argmax(mart >= 1/alpha)
                    results_apa_b[theta][N][b][eta]['rej_N'] += found
                    results_apa_b[theta][N][b][eta]['not_rej_N'] += (found==0) # should not occur        

                    # ALPHA
                    for d in dl:               
                        mart = alpha_mart(x, N, mu=1/2, eta=eta_shangrla, u=1, \
                                          estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,1,c=c,d=d))
                        # rejections by N
                        found = np.argmax(mart >= 1/alpha)
                        results_a_b[theta][N][b][eta][d]['rej_N'] += found
                        results_a_b[theta][N][b][eta][d]['not_rej_N'] += (found==0) # should not occur        

for theta in thetal:
    for N in Nl:
        for b in blanks:
            results_sqk_b[theta][N][b]['rej_N'] = results_sqk_b[theta][N][b]['rej_N']/ \
                                                  (reps-results_sqk_b[theta][N][b]['not_rej_N']) + 1
            if results_sqk_b[theta][N][b]['not_rej_N'] > 0:
                print(f'sqKelly did not reject for {theta=}, {N=}, {b=}')  # sanity check

            for eta in etal:
                results_apk_b[theta][N][b][eta]['rej_N'] = results_apk_b[theta][N][b][eta]['rej_N']/ \
                                                  (reps-results_apk_b[theta][N][b][eta]['not_rej_N']) + 1
                if results_apk_b[theta][N][b][eta]['not_rej_N'] > 0:
                    print(f'a priori Kelly did not reject for {theta=},{N=}, {b=}, {eta=}')

                results_apa_b[theta][N][b][eta]['rej_N'] = results_apa_b[theta][N][b][eta]['rej_N']/\
                                                  (reps-results_apa_b[theta][N][b][eta]['not_rej_N']) + 1
                if results_apa_b[theta][N][b][eta]['not_rej_N'] > 0:
                    print(f'a priori ALPHA did not reject for {theta=}, {N=}, {b=}, {eta=}')

                for d in dl:
                    results_a_b[theta][N][b][eta][d]['rej_N'] = results_a_b[theta][N][b][eta][d]['rej_N']/\
                                                  (reps-results_a_b[theta][N][b][eta][d]['not_rej_N']) + 1
                    if results_a_b[theta][N][b][eta][d]['not_rej_N'] > 0:
                        print(f'ALPHA did not reject for {theta=}, {N=}, {b=}, {eta=}, {d=}')

In [ ]:
print('\\begin{tabular}{lll|rrrr|rrrr|rrrr} \n' +
      '& & & \\multicolumn{4}{|c|}{$N=$10,000, \\%blank} &  \\multicolumn{4}{|c|}{$N=$100,000 \\%blank} & \\multicolumn{4}{|c}{$N=$500,000 \\%blank} \\\\ \n' +
#      ' & & & \\multicolumn{4}{|c|}{fraction blank} & \\multicolumn{4}{|c|}{fraction blank} & \\multicolumn{4}{|c}{fraction blank} \\\\ \n' +
      f'$\\theta$ & Method & params {" ".join((*["& 10 & 25 & 50 & 75 "]*3,))} \\\\'
      )

for theta in thetal:
    print(f'''\\hline {theta} & sqKelly & {" ".join(([f"& {int(results_sqk_b[theta][N][b]['rej_N']) :,.0f} " for N in Nl for b in blanks]))} \\\\''')
    for eta in etal:
        print('\\cline{2-15} &' + f''' apKelly & $\\eta=${eta} {" ".join(([f"& {int(results_apk_b[theta][N][b][eta]['rej_N']) :,.0f} " for N in Nl for b in blanks]))} \\\\''')
        for d in dl:
            print(f'''& ALPHA & $\\eta=${eta} $d=${d} {" ".join(([f"& {int(results_a_b[theta][N][b][eta][d]['rej_N']) :,.0f} " for N in Nl for b in blanks]))} \\\\''')
        print(f''' & ALPHA & $\\eta=${eta} $d=\infty$ {" ".join(([f"& {int(results_apa_b[theta][N][b][eta]['rej_N']) :,.0f} " for N in Nl for b in blanks]))} \\\\''')
        
print('\\end{tabular} \n')

In [ ]:
file_stems = ['results_sqk_b','results_apk_b','results_apa_b','results_a_b']
for fs in file_stems:
    with open(fs+'.json','w') as file:
        file.write(json.dumps(eval(fs), cls=NpEncoder, indent = 4))   

In [ ]:
best = {}
for theta in thetal:
    best[theta] = {}
    for N in Nl:
        best[theta][N] = {}
        for b in blanks:
            best[theta][N][b] = results_sqk_b[theta][N][b]['rej_N']
            for eta in etal:
                best[theta][N][b] = min([best[theta][N][b], results_apk_b[theta][N][b][eta]['rej_N'], 
                                         results_apa_b[theta][N][b][eta]['rej_N']])
                for d in dl:
                    best[theta][N][b] = min([best[theta][N][b], results_a_b[theta][N][b][eta][d]['rej_N']])
        

In [ ]:
best

In [ ]:
sqk_r = 1
apk_r = {}
a_r = {}
apa_r = {}
for eta in etal:
    apk_r[eta] = 1
    apa_r[eta] = 1
    a_r[eta] = {}
    for d in dl:
        a_r[eta][d]=1

items = 0
for theta in thetal:
    for N in Nl:
        for b in blanks:
            items += 1
            sqk_r *= results_sqk_b[theta][N][b]['rej_N']/best[theta][N][b]
            for eta in etal:
                apk_r[eta] *= results_apk_b[theta][N][b][eta]['rej_N']/best[theta][N][b]
                apa_r[eta] *= results_apa_b[theta][N][b][eta]['rej_N']/best[theta][N][b]
                for d in dl:
                    a_r[eta][d] *= results_a_b[theta][N][b][eta][d]['rej_N']/best[theta][N][b]

items

In [ ]:
print('\\begin{tabular}{llr}\\\\ \nMethod & Parameters & Score \\\\')
print(f'\\hline SqKelly & & {sqk_r**(1/items) :0.2f} \\\\ \n \hline a priori Kelly ')
for eta in etal:
    print(f' & $\\eta=${eta} & {apk_r[eta]**(1/items) :0.2f} \\\\')
print('\hline ALPHA ')
for eta in etal:
    for d in dl:
        print(f' & $\\eta=${eta} $d=${d} & {a_r[eta][d]**(1/items) :0.2f} \\\\ ')
    print(f' & $\\eta=${eta} $d=\infty$ & {apa_r[eta]**(1/items) :0.2f} \\\\')
    print('\\cline{2-3}')

print('\\end{tabular}')

## Simulations related to comparison audits

In [ ]:
# other methods
def alpha_mart(x: np.array, N: int, mu: float=1/2, eta: float=1-np.finfo(float).eps, u: float=1, \
               estim: callable=shrink_trunc) -> float :
    '''
    Finds the ALPHA martingale for the hypothesis that the population 
    mean is less than or equal to mu using a martingale method,
    for a population of size N, based on a series of draws x.

    The draws must be in random order, or the sequence is not a martingale under the null

    If N is finite, assumes the sample is drawn without replacement
    If N is infinite, assumes the sample is with replacement

    Parameters
    ----------
    x : list corresponding to the data
    N : int
        population size for sampling without replacement, or np.infinity for sampling with replacement
    mu : float in [0,u)
        hypothesized fraction of ones in the population
    eta : float in (mu,u] 
        alternative hypothesized population mean
    estim : callable
        estim(x, N, mu, eta, u) -> np.array of length len(x), the sequence of values of eta_j for ALPHA

    Returns
    -------   
    P : float
        sequentially valid p-value of the hypothesis that the population mean is less than or equal to mu
    '''
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true 
    etaj = estim(x, N, mu, eta, u) 
    with np.errstate(divide='ignore',invalid='ignore'):
        terms = np.cumprod((x*etaj/m + (1-x)*(u-etaj)/(u-m))/u)
    terms[m<0] = np.inf
    return terms

def kaplan_wald(x : np.array, N : float=np.inf, t : float=1/2, g : float=1, random_order : bool=True):
    '''
    Kaplan-Wald p-value for the hypothesis that the sample x is drawn IID from a population
    with mean t against the alternative that the mean is less than t.

    If there is a possibility that x has elements equal to zero, set g \in (0, 1); 
    otherwise, the p-value will be 1.

    If the order of the values in the sample is random, you can set random_order = True to use 
    optional stopping to increase the power. If the values are not in random order or if you want
    to use all the data, set random_order = False

    Parameters:
    -----------
    x : array-like
        the sample
    N : int / float
        population size, or np.infty for sampling with replacement
    t : float
        the null value of the mean
    g : float in [0, 1]
        protection against zeros in the sample
    random_order : Boolean
        if the sample is in random order, it is legitimate to stop early, which 
        can yield a more powerful test. See above.

    Returns:
    --------
    p-value

    '''       
    if g < 0:
        raise ValueError('g cannot be negative')
    if any(xx < 0 for xx in x):
        raise ValueError('Negative value in sample from a nonnegative population.')
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*t-S)/(N-j+1) if np.isfinite(N) else t   # mean of population after (j-1)st draw, if null is true 
    with np.errstate(divide='ignore',invalid='ignore'):
        terms = np.cumprod(g*(x/m - 1)+1)
    terms[m<0] = np.inf
    return terms    

def kaplan_kolmogorov(x : np.array, N : float=np.inf, t=1/2, g : float=0, random_order : bool=True):
    '''
    p-value for the hypothesis that the mean of a nonnegative population with N
    elements is t. The alternative is that the mean is less than t.
    If the random sample x is in the order in which the sample was drawn, it is
    legitimate to set random_order = True. 
    If not, set random_order = False. 

    g is a tuning parameter to protect against data values equal to zero.
    g should be in [0, infty)

    Parameters:
    -----------
    x : list
        observations
    N : int or np.infty
        population size, or np.inf for sampling with replacement
    t : float
        null value of the population mean
    g : float in [0, 1)
        "padding" to protect against zeros
    '''
    x = np.array(x)
    if len(x) > N:
        raise ValueError('Sample size is larger than the population!')
    if g < 0:
        raise ValueError('g cannot be negative')
    if any(xx < 0 for xx in x):
        raise ValueError('Negative value in sample from a nonnegative population.')
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*t-S)/(N-j+1) if np.isfinite(N) else t   # mean of population after (j-1)st draw, if null is true 
    with np.errstate(divide='ignore',invalid='ignore'):
        terms = np.cumprod((x+g)/(m+g))
    terms[m<0] = np.inf
    return terms

In [ ]:
# set up simulations
# first set: uniform mixed with a pointmass at 1

reps = int(10**4)
alpha = 0.05
mixtures = [.99, .9, .75, .5, .25, .1, .01]  # mass at 1
zero_mass = [0, 0.001] # mass at 0

al = {}  # alpha martingale
kw = {}  # Kaplan-Wald
kk = {}  # Kaplan-Kolmogorov
apk = {} # a priori Kelly
sqk = {} # square Kelly
thetas = {}

methods = [al, kw, kk, apk, sqk, thetas]

g_kol = [0.01, 0.1, 0.2]  # for the Kaplan-Kolmogorov method
g_wald = 1 - np.array(g_kol) # for Kaplan-Wald method
    
D = 10 # for APK
beta = 1 
dl = [10, 100]        # for alpha
c_base = 0.5          # for alpha. larger c since there is no particular expectation about error rates
etal = [.99, .9, .75, .55]
Nl = [10000, 100000, 500000]


for m in mixtures:
    for meth in methods:
        meth[m] = {}
        for N in Nl:
            meth[m][N] = {}

zm = zero_mass[1]
for m in mixtures:
    print(f'{m=}')
    for N in Nl:
        print(f'\t{N=}')
        sqk[m][N]=0
        thetas[m][N]=0
        for eta in etal:
            apk[m][N][eta] = 0
            al[m][N][eta] = {}
            for d in dl:
                al[m][N][eta][d] = 0
        for g in g_kol:
            kk[m][N][g] = 0
        for g in g_wald:
            kw[m][N][g] = 0 
        t = 0
        while t <= 0.5:
            x = sp.stats.uniform.rvs(size=N)
            y = sp.stats.uniform.rvs(size=N)
            x[y<=m] = 1
            x[y>=(1-zm)] = 0
            t = np.mean(x)
        thetas[m][N] = t
        for i in range(reps):
            np.random.shuffle(x)
            mart = sqKelly_martingale(x, m=1/2, N=N, D=D, beta=beta)
            sqk[m][N] += np.argmax(mart >= 1/alpha)
            for g in g_kol:
                mart = kaplan_kolmogorov(x, N, t=1/2, g=g)
                kk[m][N][g] += np.argmax(mart >= 1/alpha)
            for g in g_wald:
                mart = kaplan_wald(x, N, t=1/2, g=g)                
                kw[m][N][g] += np.argmax(mart >= 1/alpha)
            for eta in etal:
                mart = apriori_Kelly_martingale(x, m=0.5, N=N, n_A=int(N*eta), n_B=N-int(N*eta))
                apk[m][N][eta] += np.argmax(mart >= 1/alpha)
                c = c_base*(eta-1/2)
                for d in dl:
                    mart = alpha_mart(x, N, mu=1/2, eta=eta, u=1, \
                                estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,1,c=c,d=d))
                    al[m][N][eta][d] += np.argmax(mart >= 1/alpha)

In [ ]:
for m in mixtures:
    for N in Nl:
        sqk[m][N] = sqk[m][N]/reps + 1
        for eta in etal:
            apk[m][N][eta] = apk[m][N][eta]/reps + 1
            for d in dl:
                al[m][N][eta][d] = al[m][N][eta][d]/reps + 1
        for g in g_kol:
            kk[m][N][g] = kk[m][N][g]/reps + 1
        for g in g_wald:
            kw[m][N][g] =  kw[m][N][g]/reps +1

In [ ]:
file_stems = ['al', 'kw', 'kk', 'apk', 'sqk', 'thetas']
for fs in file_stems:
    with open(fs+'.json','w') as file:
        file.write(json.dumps(eval(fs), cls=NpEncoder, indent = 4))   

In [ ]:
print('\\begin{tabular}{lll|rrr} \n mass at 1 & method & params & $N=$10,000 &  $N=$100,000 & $N=$500,000 \\\\')
for m in mixtures:
    print(f'''\\hline {m :.2f} & sqKelly & {" ".join(([f"& {sqk[m][N] :,.0f} " for N in Nl]))} \\\\''')
    for eta in etal:
        print('\\cline{2-6} &' + f''' apKelly & $\\eta=${eta} {" ".join(([f"& {apk[m][N][eta] :,.0f} " for N in Nl]))} \\\\''')
        print('\\cline{2-6}')
        for d in dl:
            print(f'''& ALPHA & $\\eta=${eta} $d=${d} {" ".join(([f"& {al[m][N][eta][d] :,.0f} " for N in Nl]))} \\\\''')  
    print('\\cline{2-6}')
    for g in g_kol:
        print(f''' & Kaplan-Kolmogorov & {g=} {" ".join(([f"& {kk[m][N][g] :,.0f} " for N in Nl]))} \\\\''')       
    print('\\cline{2-6}')
    for g in g_wald:
        print(f''' & Kaplan-Wald & {g=} {" ".join(([f"& {kw[m][N][g] :,.0f} " for N in Nl]))} \\\\''')         
 
print('\\end{tabular} \n')
print('\\caption{\\protect \\label{tab:comparison-1} Mean sample sizes to reject the hypothesis that ' + 
      f'the mean is less than $1/2$ at significance level ${alpha :.2f}$ for various methods, in {reps :,.0f} ' +
      f' simulations with mass {zm :.3f} zero, mass $m$ at 1, and mass $1-m-{zm :0.3f}$ uniformly ' +
      ' distributed on $[0, 1]$, for values of $m$ between 0.99 and 0.5. The smallest mean sample size ' +
      'for each combination of $m$ and $N$ is in bold font.}')

In [ ]:
best = {}
for m in mixtures:
    best[m] = {}
    for N in Nl:
        best[m][N] = sqk[m][N]
        for eta in etal:
            best[m][N] = min(best[m][N], apk[m][N][eta])
            for d in dl:
                best[m][N] = min(best[m][N], al[m][N][eta][d])
        for g in g_kol:
            best[m][N] = min(best[m][N], kk[m][N][g])
        for g in g_wald:
            best[m][N] = min(best[m][N], kw[m][N][g])

best

In [ ]:
sqk_r = 1
apk_r = {}
al_r = {}
kk_r = {}
kw_r = {}
for eta in etal:
    apk_r[eta] = 1
    al_r[eta] = {}
    for d in dl:
        al_r[eta][d]=1
for g in g_kol:
    kk_r[g] = 1
for g in g_wald:
    kw_r[g] = 1

items = 0

for m in mixtures:
    for N in Nl:
        items += 1
        sqk_r *= sqk[m][N]/best[m][N]
        for eta in etal:
            apk_r[eta] *= apk[m][N][eta]/best[m][N]
            for d in dl:
                al_r[eta][d] *= al[m][N][eta][d]/best[m][N]
        for g in g_kol:
            kk_r[g] *= kk[m][N][g]/best[m][N]
        for g in g_wald:
            kw_r[g] *= kw[m][N][g]/best[m][N]

items

In [ ]:
print('\\begin{tabular}{llr}\\\\ \nMethod & Parameters & Score \\\\')
print(f'\\hline SqKelly & & {sqk_r**(1/items) :0.2f} \\\\ \n \hline a priori Kelly ')
for eta in etal:
    print(f' & $\\eta=${eta} & {apk_r[eta]**(1/items) :0.2f} \\\\')
print('\hline ALPHA ')
for eta in etal:
    for d in dl:
        print(f' & $\\eta=${eta} $d=${d} & {al_r[eta][d]**(1/items) :0.2f} \\\\ ')
    print('\\cline{2-3}')
print ('\hline Kaplan-Kolmogorov')
for g in g_kol:
    print(f' & $g=${g} & {kk_r[g]**(1/items) :0.2f}\\\\ ')
print ('\hline Kaplan-Wald')
for g in g_wald:
    print(f' & $g=${g} & {kw_r[g]**(1/items) :0.2f}\\\\ ')


print('\\end{tabular}')

## Calculations to make Huang et al. table 2 commensurable

In [ ]:
v = [[17, 93, 100, 100, 1785,  864, 198,  95,  44], \
     [ 6, 89, 100, 100, 1985, 1505, 760, 542, 377], \
     [37, 98, 100, 100, 1561,  572, 200, 131,  86], \
     [ 8, 20,  83, 100, 1846, 1621, 552,  99,  38], \
     [34, 98, 100, 100, 1618,  628, 167,  88,  45]]

full = 18000
def increment(v):
    for i in range(4):
        v[i+4] = int(v[i+4]+(100-v[i])*full/100)
    print(f'''{" ".join([f"& {v[i] :,.0f} " for i in range(4,9)])} \\\\''')

for vv in v:
    increment(vv)

In [ ]:
a